<a href="https://colab.research.google.com/github/johanhoffman/DD2363-VT19/blob/maxbergmark/Lab-3/maxbergmark_lab3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Lab 3: Iterative methods**
**Max Bergmark**

# **Abstract**

This lab is about implementing iterative methods to solve matrix equations and find zeros of functions. 

#**About the code**

I am the author of the code in its entirety.

# **Set up environment**

To have access to the neccessary modules you have to run this cell. If you need additional modules, this is where you add them. 

In [0]:
# Load neccessary modules.
from google.colab import files

import numpy as np

# **Methods**

## Helper functions

To improve readability of the code, I implemented these two helper functions.

In [0]:
# return a normalized version of a vector
def normalize(v):
    return v / np.linalg.norm(v)
  
# return the length of a vector
def norm(v):
    return np.linalg.norm(v)

## 1: Jacobi iteration for $Ax=b$

The idea behind Jacobi iteration is to split the matrix A into a diagonal matrix and a residual. Since it is easy to find the inverse of a diagonal matrix, the iteration step becomes trivial. Once the inverse is found, we iterate 1000 times and then return the found vector $x$. For matrix equations with exact solutions, you could add a breaking condition. However, if the system is not exactly solvable, it is more difficult to find a stopping criterion.

In [0]:
def jacobi_iterate(A, b):
    D = np.diag(np.diag(A))
    R = A - D
    D_inv = np.diag(1/np.diag(D))
    x = np.ones(A.shape[1])
    for _ in range(1000):
        x = np.dot(D_inv, b - np.dot(R, x))
    return x


## 2: Gauss-Seidel iteration for $Ax=b$

The Gauss-Seidel iteration is also based around splitting the matrix $A$ into two parts, but this time it is a lower triangular part, and the residual. The inverse of a triangular matrix is relatively easy to find, which makes this algorithm easy to implement. 

In [0]:
def gauss_seidel(A, b):
    x = np.zeros(A.shape[1])
    for _ in range(1000):
        x_new = x.copy()
        for i in range(x.size):
            L_sum = np.dot(A[i,:i], x_new[:i])
            U_sum = np.dot(A[i,i+1:], x[i+1:])
            x_new[i] = 1/A[i,i]*(b[i] - L_sum - U_sum)
            x = x_new
    return x_new

## 3: Newton's method for scalar nonlinear equation $f(x)=0$

Newton's method for finding zeros of functions. Geometrically, it works by starting at any function point, and drawing a tangent along the function and checking where that tangent intersects the x axis. The x-value for the intersection point becomes your new guess.

To implement this algorithm, you must be able to calculate the derivative of the function at certain points. This can be done by using central difference, which has better convergence than the regular formula for function derivatives. 

In [0]:
def df(f, x, h):
    return (f(x+h) - f(x-h)) / (2*h)

def newton(f, x):
    iters = 0
    fx = f(x)
    while np.max(abs(fx)) > 1e-15:
        fx = f(x)
        x -= fx / df(f, x, 1e-8)
        iters += 1
        if iters > 100000:
            print(f(x))
            raise ValueError("No solution found")
    return x

## 4: GMRES method for $Ax=b$

The Generalized Minimal Residuals method is another method for solving matrix equations. It is more complicated, and since the algorithm itself also includes solving a least-squares problem, it feels superfluous. 

It works by applying successive Arnoldi iterations, and then minimizing the residual. 

In [0]:
def arnoldi(A, b, k, Q, H, k_max):
    v = np.dot(A, Q[:,k])
    for j in range(k):
        H[j,k] = np.dot(Q[:,j], v)
        v -= H[j,k]*Q[:,j]
    H[k+1,k] = norm(v)
    if (H[k+1, k] != 0 and k != k_max -1):
        Q[:,k+1] = v / H[k+1,k]

def gmres(A, b):
    k_max = 100
    n = A.shape[0]
    x0 = np.zeros(n)

    x = np.zeros(n)
    r = b - np.dot(A, x0)

    Q = np.zeros((n, k_max))
    H = np.zeros((k_max+1, k_max))
    Q[:,0] = normalize(r)

    for k in range(k_max):
        arnoldi(A, b, k, Q, H, k_max)

        b = np.zeros(k_max+1)
        b[0] = norm(r)

        res = np.linalg.lstsq(H, b, rcond = None)[0]
        x = np.dot(Q, res) + x0

    return x

## 5: Newton's method for vector nonlinear equation $f(x)=0$

This task uses the exact same code as the one for task 3, since I implemented it to handle scalars implicitly while handling numpy arrays. In higher dimensions, the gradient is the analog to the derivative. It is computed in the exact same way for each dimension as the 1-dimensional case. 

# **Results**

## 1: Jacobi iteration for $Ax=b$

In [7]:
def test_jacobi_iteration():
    A = np.array([[2, 1], [5, 7]], dtype = np.float64)
    b = np.array([11, 13])
    x_true = np.array([7+1/9, -3-2/9])
    x = jacobi_iterate(A, b)
    assert np.allclose(x, x_true)
    A = np.array(
        [
            [10, -1, 2, 0], 
            [-1, 11, -1, 3], 
            [2, -1, 10, -1], 
            [0, 3, -1, 8]
        ], dtype = np.float64
    )
    b = np.array([6, 25, -11, 15])
    x_true = np.array([1, 2, -1, 1])
    x = jacobi_iterate(A, b)
    assert np.allclose(x, x_true)

test_jacobi_iteration()
print("Test passed!")

Test passed!


## 2: Gauss-Seidel iteration for $Ax=b$


In [8]:
def test_gauss_seidel():
    A = np.array([[16, 3], [7, -11]], dtype = np.float64)
    b = np.array([11, 13])
    x_true = np.array([0.81218274, -0.66497462])
    x = gauss_seidel(A, b)
    assert np.allclose(x, x_true)
    A = np.array([[2, 1], [5, 7]], dtype = np.float64)
    b = np.array([11, 13])
    x_true = np.array([7+1/9, -3-2/9])
    x = gauss_seidel(A, b)
    assert np.allclose(x, x_true)
    
test_gauss_seidel()
print("Test passed!")

Test passed!


## 3: Newton's method for scalar nonlinear equation $f(x)=0$

Here, we test 100 random polynomials of degree 2. Since we can determine the exact solutions, we are able to compare the found value to the exact values, and verify that it converges to either one of them. 

In [9]:
def test_polynomial():
    for _ in range(100):
        p = np.random.randn()
        q = np.random.randn()
        f = lambda x: x**2 + p*x + q
        if q < p*p/4:
            x_0 = -p/2 + (p*p/4 - q)**.5
            x_1 = -p/2 - (p*p/4 - q)**.5
            x_start = 0
            x = newton(f, x_start)
            assert np.isclose(x, x_0) or np.isclose(x, x_1)
            
test_polynomial()
print("Test passed!")

Test passed!


## 4: GMRES method for $Ax=b$

In [13]:
def test_GMRES():
    A = np.array([[2, 1], [5, 7]], dtype = np.float64)
    b = np.array([11, 13])
    x_true = np.array([7+1/9, -3-2/9])
    # x = GMRes(A, b, np.zeros(A.shape[1]), 0, 5)
    # print(x)
    x = gmres(A, b)
    assert np.allclose(x, x_true)
    A = np.array([[16, 3], [7, -11]], dtype = np.float64)
    b = np.array([11, 13])
    x_true = np.array([0.81218274, -0.66497462])
    x = gmres(A, b)
    assert np.allclose(x, x_true)
    A = np.array([[2, 1], [5, 7]], dtype = np.float64)
    b = np.array([11, 13])
    x_true = np.array([7+1/9, -3-2/9])
    x = gmres(A, b)
    assert np.allclose(x, x_true)
    
test_GMRES()
print("Test passed!")

Test passed!


## 5: Newton's method for vector nonlinear equation $f(x)=0$

Here, we test the vector function $(x-1)^2$, which has the solution $x_i = 1$. We also test the function $g(x)$, which is designed to have the solution $x_i = i$. We test them for different number dimensions between 1 and 10.

In [11]:
def test_vector_polynomial():
    f = lambda x: (x - 1)**2 
    g = lambda x: (x- np.arange(x.size))**3
    for n in range(1, 10):
        x = np.zeros(n)
        x = newton(f, x)
        assert np.allclose(x, 1)
        x = np.zeros(n)
        x = newton(g, x)        
        assert np.allclose(x, np.arange(x.size))

test_vector_polynomial()
print("Test passed!")

Test passed!


## Full test suite

Use this to run all tests at once.

In [12]:
def run_test_suite():
    test_jacobi_iteration()
    test_gauss_seidel()
    test_polynomial()
    test_GMRES()
    test_vector_polynomial()

run_test_suite()
print("All tests passed!")

All tests passed!


# **Discussion**

All of the algorithms were fairly straight-forward to implement given the instructions in the lecture notes. However, the GMRES algorithm wouldn't converge properly when I used the least squares solver from Lab 2, so I used the one from numpy instead. 